In [1]:
import fitz
import pandas as pd
import re
from operator import itemgetter

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import numpy as np
# Creating Understanding of whole pdf font details and size by page
doc = fitz.open('5473.pdf')
font_counts = {}

for page_num in range(0,doc.pageCount):
    blocks = doc[page_num].getText("dict")["blocks"]
    for b in blocks:  # iterate through the text blocks
        if b['type'] == 0:
            for l in b["lines"]:  # iterate through the text lines
                for s in l["spans"]:  # iterate through the text spans
                    identifier = "{0}^{1}^{2}^{3}".format(page_num, s['size'], s['flags'], s['font'])
                    font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usag
                    
font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

# Extracting Most Used font and size combo in each page

total_read=0

mst_usd_pg_fnt = []

for page_num in range(0,doc.pageCount):
    flg=False
    for itm in font_counts:
        if int(itm[0].split("^")[0]) == page_num and flg == False:
            mst_usd_pg_fnt.append(itm[0])
            flg=True


number_of_pages = doc.pageCount

df = pd.DataFrame(columns=['LineText','BlockwiseNumber','VerticalSpaceLength','IsHorizontalTab','IsStartCap','IsEndDot','IsStartSpace','NormSize',
                                       'DocPageNumber','FontWeight','LineLength','LineIdentifier','identifier'])
def flags_decomposer(flags):
    """Make font flags human readable."""
    l = 0
   
    if flags & 2 ** 1:
        l = 1

    if flags & 2 ** 4:
        l = 2
        
    return l

for page_number in range(number_of_pages):
    page = doc.loadPage(page_number)
    blocks = page.getText("dict")["blocks"]#rawdict")
    lists={}
    previous_id=''
    previous_text=''
    vertical_line_length=0.0
    p_space=0.0
    lists=[]
    count=0
    prev_blocknum=0.00
    prev_bbox3num=0.00
    for b in blocks:
        block_num = b["bbox"][3]
        if b['type'] == 0:
        
            for idx,l in enumerate(b["lines"]):
               
                for s in l["spans"]:
                    if s["text"].startswith(' ') == True:#not required starts with
                        a_space=1
                    else:
                        a_space=0
                     
                    if re.match(r'[ \t]',s["text"]) == True:
                        a_horizontaltab=1
                    else:
                        a_horizontaltab=0
                
                    if(idx==previous_id) and (idx!=0) and (previous_id!=0):
                        s["text"]=previous_text + s["text"]
                    
                    v_len=s["bbox"][3]-vertical_line_length
                
                    append=0
                    if(idx==previous_id) and ((s["bbox"][3]-vertical_line_length)<0 and (s["bbox"][3]-vertical_line_length)>-1) :
                        if previous_text not in s["text"]:
                            s["text"]=previous_text + s["text"]

                    startchar=s["text"].replace("'", "")
            
                    startchar=startchar.strip('"')
                    startchar=startchar.strip()
               
                    if startchar[:1].isupper():#not required starts with
                        startcaps=1
                    else:
                        startcaps=0
                    if startchar[:1].isdigit():
                        start_digit=1
                    else:
                        start_digit=0

                    if startchar[:1] == '*':
                        start_asterisk=1
                    else:
                        start_asterisk=0

                    if startchar[:1] == '"':
                        start_parenthesis=1
                    else:
                        start_parenthesis=0

                    if s["text"].strip().endswith('.') == True:#not required starts with
                        endwithdot=1
                    else:
                        endwithdot=0
                    if len(s["text"])>1:
                        size= s["size"]
                    else:
                        size=0
                    
                    if len(s["text"])>1:
                        flags=flags_decomposer(s["flags"])
                    else:
                        flags =''
                    
                    lists.append([page_number,previous_id,idx,previous_text,s["text"],v_len,startcaps,endwithdot,a_space,
                                  a_horizontaltab,size,flags,"{0}^{1}^{2}^{3}´{4}".format(page_number, s['size'], s['flags'], s['font'], s['bbox']),start_digit,start_asterisk,start_parenthesis,s['bbox'][3],
                                  block_num])
                    if  len(s["text"].strip())>0: #and (s["bbox"][3]
                        vertical_line_length=s["bbox"][3]
  
                    if  len(s["text"].strip())>0:
                        p_space=v_len#s["bbox"][3]-vertical_line_length

                    previous_id=idx
                
                    previous_text=s["text"] 
                    prev_bbox3num=s['bbox'][3]
                    prev_blocknum=block_num
                    # print(block_num)
    
    vertical_len_pre=0.0

    final_list=[]

    dupl = []
    for index,x in enumerate(lists):
        
        if x[4] not in dupl:
            
            a_row=pd.Series([x[4],x[2],x[5],x[9],x[6],x[7],x[8],x[10],x[0],x[11],len(x[4]),x[12],x[13],x[14],x[15],x[16],x[17]],
                                index=['LineText','BlockwiseNumber','VerticalSpaceLength','IsHorizontalTab','IsStartCap','IsEndDot','IsStartSpace','NormSize',
                                       'DocPageNumber','FontWeight','LineLength','LineIdentifier','start_digit','start_asterisk','start_parenthesis','sbox3','blocknum'])
            row_df = pd.DataFrame([a_row])
            df=df.append(row_df)

    

df['Identifier'] =df.index+1

df['LineText'].replace(' ', np.nan, inplace=True)
df.dropna(subset=['LineText'], inplace=True)
df[['Label','LineBoundingBox']] = df.LineIdentifier.str.split("´",expand=True)
df['LineBoundingBox'] = df['LineBoundingBox'].str.replace('(','')
df['LineBoundingBox'] = df['LineBoundingBox'].str.replace(')','')
df[['LtoR','DtoU','RtoL','UtoD']] = df.LineBoundingBox.str.split(",",expand=True)

df['LtoR'] = df['LtoR'].astype(float)
df['DtoU'] = df['DtoU'].astype(float)
df['UtoD'] = df['UtoD'].astype(float)
df['RtoL'] = df['RtoL'].astype(float)


df['DtoU_last']=""
df['DtoU_next']=""
df['LtoR_last']=""
df['LtoR_next']=""
df['RtoL_last']=""
df['RtoL_next']=""
df['UtoD_next']=""
df['UtoD_last']=""

for i in range(len(df)):
    if i<len(df)-1:
          df['LtoR_next'].values[i]=df['LtoR'].values[i+1] - df['LtoR'].values[i]
          df['UtoD_next'].values[i]=df['UtoD'].values[i+1] - df['UtoD'].values[i]
          df['DtoU_next'].values[i]=df['DtoU'].values[i+1] - df['DtoU'].values[i]
          df['RtoL_next'].values[i]=df['RtoL'].values[i+1] - df['RtoL'].values[i]
    if i!=0 :
          df['DtoU_last'].values[i]=df['DtoU'].values[i] - df['DtoU'].values[i-1]
          df['LtoR_last'].values[i]=df['LtoR'].values[i] - df['LtoR'].values[i-1]
          df['RtoL_last'].values[i]=df['RtoL'].values[i] - df['RtoL'].values[i-1]
          df['UtoD_last'].values[i]=df['UtoD'].values[i] - df['UtoD'].values[i-1]

df['LtoR']=pd.to_numeric(df['LtoR'].fillna(0))
df['DtoU']=pd.to_numeric(df['DtoU'].fillna(0))
df['RtoL']=pd.to_numeric(df['RtoL'].fillna(0))
df['UtoD']=pd.to_numeric(df['UtoD'].fillna(0))
df['LtoR_last']=pd.to_numeric(df['LtoR_last'].fillna(0))
df['LtoR_next']=pd.to_numeric(df['LtoR_next'].fillna(0))
df['DtoU_next']=pd.to_numeric(df['DtoU_next'].fillna(0))
df['DtoU_last']=pd.to_numeric(df['DtoU_last'].fillna(0))
df['UtoD_last']=pd.to_numeric(df['UtoD_last'].fillna(0))
df['UtoD_next']=pd.to_numeric(df['UtoD_next'].fillna(0))
df['RtoL_last']=pd.to_numeric(df['RtoL_last'].fillna(0))
df['RtoL_next']=pd.to_numeric(df['RtoL_next'].fillna(0))




df['FontWeight'] = df['FontWeight'].fillna(0)




#calculate min and max
min_LtoR = np.min(df['LtoR'])
min_DtoU = np.min(df['DtoU'])
min_UtoD = np.min(df['UtoD'])
min_RtoL = np.min(df['RtoL'])
min_LtoR_last=np.min(df['LtoR_last'])
min_LtoR_next=np.min(df['LtoR_next'])
min_DtoU_next=np.min(df['DtoU_next'])
min_DtoU_last=np.min(df['DtoU_last'])
min_UtoD_last=np.min(df['UtoD_last'])
min_UtoD_next=np.min(df['UtoD_next'])
min_RtoL_last=np.min(df['RtoL_last'])
min_RtoL_next=np.min(df['RtoL_next'])
min_ver_length= np.min(df['VerticalSpaceLength'])


max_LtoR= np.max(df['LtoR'])
max_DtoU=np.max(df['DtoU'])
max_UtoD=np.max(df['UtoD'])
max_RtoL=np.max(df['RtoL'])
max_LtoR_last=np.max(df['LtoR_last'])
max_LtoR_next=np.max(df['LtoR_next'])
max_DtoU_last=np.max(df['DtoU_last'])
max_DtoU_next=np.max(df['DtoU_next'])
max_UtoD_last=np.max(df['UtoD_last'])
max_UtoD_next=np.max(df['UtoD_next'])
max_RtoL_last=np.max(df['RtoL_last'])
max_RtoL_next=np.max(df['RtoL_next'])
max_ver_length= np.max(df['VerticalSpaceLength'])
min_normsize=np.min(df['NormSize'])
max_normsize=np.max(df['NormSize'])

df['FontWeight'] = df['FontWeight'].fillna(0.00)

df.loc[df['BlockwiseNumber'] > 1, 'BlockwiseNumber'] = 1

def numberofcaps(df):
    no_of_caps = sum(1 for c in df['LineText'] if c.isupper())
    return no_of_caps

def numberofcommas(df):
    no_of_commas = sum(1 for c in df['LineText'].strip() if c == ',')
    return no_of_commas

def endwithcomma(df):
    endswithcomma = 0
    if df['LineText'].strip().endswith(',') == True:
        endswithcomma = 1
    return endswithcomma

# message= "In contrast to possible Changes In the National context"
df['Num_of_caps'] = df.apply (lambda row: numberofcaps(row), axis=1)
df['Ends_with_comma'] = df.apply (lambda row: endwithcomma(row), axis=1)
df['Number_of_commas'] = df.apply (lambda row: numberofcommas(row), axis=1)

min_Num_of_caps=np.min(df['Num_of_caps'])
max_Num_of_caps=np.max(df['Num_of_caps'])

min_Number_of_commas=np.min(df['Number_of_commas'])
max_Number_of_commas=np.max(df['Number_of_commas'])


min_LineLength=np.min(df['LineLength'])
max_LineLength=np.max(df['LineLength'])

#removed minmax for few columns as values are going to e power values and effectig autoencoder

for i in range(len(df)):
  #  i = j+1
    df['LtoR'].values[i]=(df['LtoR'].values[i] - min_LtoR) / (max_LtoR - min_LtoR)
    df['DtoU'].values[i]=(df['DtoU'].values[i] - min_DtoU) / (max_DtoU - min_DtoU)
    df['UtoD'].values[i]=(df['UtoD'].values[i] - min_UtoD) / (max_UtoD - min_UtoD)
    df['RtoL'].values[i]=(df['RtoL'].values[i]  - min_RtoL) / (max_RtoL - min_RtoL)
    df['LineLength'].values[i]=((df['LineLength'].values[i]  - min_LineLength) / (max_LineLength - min_LineLength))
    df['LtoR_last'].values[i]=(df['LtoR_last'].values[i] - min_LtoR_last) / (max_LtoR_last - min_LtoR_last)
    df['LtoR_next'].values[i]=(df['LtoR_next'].values[i] - min_LtoR_next) / (max_LtoR_next - min_LtoR_next)
    df['DtoU_last'].values[i]=(df['DtoU_last'].values[i] - min_DtoU_last) / (max_DtoU_last - min_DtoU_last)
    df['DtoU_next'].values[i]=(df['DtoU_next'].values[i] - min_DtoU_next) / (max_DtoU_next - min_DtoU_next)
    df['UtoD_last'].values[i]=(df['UtoD_last'].values[i] - min_UtoD_last) / (max_UtoD_last - min_UtoD_last)
    df['UtoD_next'].values[i]=(df['UtoD_next'].values[i] - min_UtoD_next) / (max_UtoD_next - min_UtoD_next)
    df['RtoL_last'].values[i]=(df['RtoL_last'].values[i] - min_RtoL_last) / (max_RtoL_last - min_RtoL_last)
    df['RtoL_next'].values[i]=(df['RtoL_next'].values[i] - min_RtoL_next) / (max_RtoL_next - min_RtoL_next)
    df['VerticalSpaceLength'].values[i]=((df['VerticalSpaceLength'].values[i]  - min_ver_length) / (max_ver_length - min_ver_length))
    df['NormSize'].values[i]=((df['NormSize'].values[i]  - min_normsize) / (max_normsize - min_normsize))
  #df['Number_of_commas'].values[i]=((df['Number_of_commas'].values[i]  - min_Number_of_commas) / (max_Number_of_commas - min_Number_of_commas))
  #df['Num_of_caps'].values[i]=((df['Num_of_caps'].values[i]  - min_Num_of_caps) / (max_Num_of_caps - min_Num_of_caps))




#Calculate Median 
median_LtoR_next= df['LtoR_next'].median()
median_LtoR_last= df['LtoR_last'].median()
median_DtoU_next= df['DtoU_next'].median()
median_DtoU_last= df['DtoU_last'].median()
median_UtoD_last= df['UtoD_last'].median()
median_UtoD_next= df['UtoD_next'].median()
median_RtoL_last= df['RtoL_last'].median()
median_RtoL_next= df['RtoL_next'].median()
median_Number_of_commas= df['Number_of_commas'].median()
median_Num_of_caps= df['Num_of_caps'].median()



for i in range(len(df)):
    if df['LtoR_last'].values[i]<0:
        df['LtoR_last'].values[i]=median_LtoR_last
            
    if df['LtoR_next'].values[i]<0:
        df['LtoR_next'].values[i]=median_LtoR_next
        
    if df['DtoU_next'].values[i]<0:
        df['DtoU_next'].values[i]=median_DtoU_next

    if df['DtoU_last'].values[i]<0:
        df['DtoU_last'].values[i]=median_DtoU_last
        
    if df['UtoD_last'].values[i]<0:
        df['UtoD_last'].values[i]=median_UtoD_last
        
    if df['UtoD_next'].values[i]<0:
        df['UtoD_next'].values[i]=median_UtoD_next
        
    if df['RtoL_last'].values[i]<0:
        df['RtoL_last'].values[i]=median_RtoL_last

    if df['RtoL_next'].values[i]<0:
        df['RtoL_next'].values[i]=median_RtoL_next     
     
    if df['Number_of_commas'].values[i]<0:
        df['Number_of_commas'].values[i]=median_Number_of_commas

    if df['Num_of_caps'].values[i]<0:
        df['Num_of_caps'].values[i]=median_Num_of_caps


# df_result=df.drop_duplicates(subset ="LineText",keep = 'first') 
df.reset_index(level=0, inplace=True) 

df= df.fillna(0)


# df_result= df_result.drop(columns=['index','Identifier'])
df['Identifier'] = df.index+1

 
text_length=15 

df_features=df[['Identifier','LineText','BlockwiseNumber', 'VerticalSpaceLength','IsStartCap', 'IsEndDot', 'IsStartSpace','NormSize',
                       'FontWeight', 'LineLength','start_digit', 'start_asterisk', 'start_parenthesis','LtoR', 'DtoU', 'RtoL', 'UtoD', 'DtoU_last',
                        'DtoU_next', 'LtoR_last', 'LtoR_next', 'RtoL_last', 'RtoL_next', 'UtoD_next', 'UtoD_last','Num_of_caps','Ends_with_comma','Number_of_commas']]
df_features.to_csv('features_4.csv')
df_features.head(200)

,Identifier,LineText,BlockwiseNumber,VerticalSpaceLength,IsStartCap,IsEndDot,IsStartSpace,NormSize,FontWeight,LineLength,start_digit,start_asterisk,start_parenthesis,LtoR,DtoU,RtoL,UtoD,DtoU_last,DtoU_next,LtoR_last,LtoR_next,RtoL_last,RtoL_next,UtoD_next,UtoD_last,Num_of_caps,Ends_with_comma,Number_of_commas
0,1,Parliaments in Central and Eastern Europe:,0,0.571645,1,0,0,0.923077,2,0.446809,0.0,0.0,0.0,0.240369,0.069231,0.830732,0.077368,0.000000,0.513421,0.000000,0.548018,0.000000,0.454212,0.513414,0.000000,4,0,0
1,2,Changing Legislative Institutions*,1,0.504063,1,0,0,0.923077,2,0.351064,0.0,0.0,0.0,0.301358,0.096074,0.762385,0.104196,0.513421,0.526540,0.548018,0.590698,0.454212,0.416300,0.522166,0.513414,3,0,0
2,3,PETR KOPECKÝ**,0,0.512982,1,0,0,0.615385,2,0.138298,0.0,0.0,0.0,0.443738,0.149154,0.621726,0.148528,0.526540,0.509860,0.590698,0.430009,0.416300,0.575053,0.509848,0.522166,11,0,0
3,4,"Department of Political Science, Leiden Univer...",1,0.500427,1,0,0,0.615385,0,0.521277,0.0,0.0,0.0,0.279683,0.168874,0.783869,0.168223,0.509860,0.522862,0.430009,0.435528,0.575053,0.186270,0.523947,0.509848,5,0,1
4,5,Abstract,0,0.514798,1,0,0,0.692308,2,0.074468,0.0,0.0,0.0,0.126153,0.214597,0.204458,0.216116,0.522862,0.499984,0.435528,0.562260,0.186270,0.873331,0.499976,0.523947,1,0,0
5,6,Abstract. Parliaments emerged as one of the ke...,0,0.490366,1,0,0,0.692308,0,0.872340,0.0,0.0,0.0,0.214301,0.214565,0.935525,0.216069,0.499984,0.509608,0.562260,0.469813,0.873331,0.490064,0.509603,0.499976,2,0,0
6,7,communist transition. Endowed with the signifi...,1,0.500178,0,0,0,0.692308,0,0.797872,0.0,0.0,0.0,0.126153,0.233781,0.935561,0.235274,0.509608,0.509608,0.469813,0.516036,0.490064,0.490086,0.509603,0.509603,1,0,0
7,8,"the old communist constitutions, they were ass...",1,0.500178,0,0,0,0.692308,0,0.851064,0.0,0.0,0.0,0.126153,0.252997,0.935641,0.254480,0.509608,0.509608,0.516036,0.516036,0.490086,0.490030,0.509603,0.509603,0,0,1
8,9,ifying the new democratic constitutions in the...,1,0.500178,0,0,0,0.692308,0,0.840426,0.0,0.0,0.0,0.126153,0.272213,0.935612,0.273686,0.509608,0.509608,0.516036,0.516036,0.490030,0.490066,0.509603,0.509603,1,0,0
9,10,"er institutions in Central and Eastern Europe,...",1,0.500178,0,0,0,0.692308,0,0.808511,0.0,0.0,0.0,0.126153,0.291428,0.935652,0.292891,0.509608,0.509608,0.516036,0.516036,0.490066,0.489995,0.509603,0.509603,3,0,1
